<a href="https://colab.research.google.com/github/eshrinivasan/collab-files/blob/main/pretrain_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!npx degit https://github.com/eshrinivasan/collab-files -f
!pip install transformers
!pip install pytorch_lightning

npx: installed 1 in 1.513s
> destination directory is not empty. Using --force, continuing
> cloned eshrinivasan/collab-files#HEAD


In [ ]:
import math
import pickle
import random
import multiprocessing as mp
from transformers import BertTokenizer
from tqdm import tqdm

flatten = lambda t: [item for sublist in t for item in sublist]

tok = BertTokenizer.from_pretrained("bert-large-uncased")

BATCH_SIZE = 128

def tokenize(x):
    return tok.encode(x)[1:-1]

def process(_n):
    n = _n
    size = math.ceil(len(n)*0.15)
    indices = random.sample(range(len(n)), size)
    replacement_prob = [True, True, True, True, True, True, True, True, False, False]
    output = [-100]*len(n)
    for index in indices:
        output[index] = n[index]
        if random.choice(replacement_prob):
            n[index] = 103
        else:
            if random.choice([True, False]):
                n[index] = random.randint(1010, 30500)
    return (n, output)

def batcher(ins, outs):
    batchx, batchy, batchesx, batchesy, batchesmasks = [], [], [], [], []
    for i in zip(ins, outs):
        batchx.append(i[0])
        batchy.append(i[1])
        if len(batchx) == BATCH_SIZE:
            maxlen = max([len(x) for x in batchx]) * 3
            masks = [[1] * len(x) + [0] * (maxlen - len(x)) for x in batchx]
            batchx = [x + [0] * (maxlen - len(x)) for x in batchx]
            batchy = [x + [0] * (maxlen - len(x)) for x in batchy]
            batchesx.append(batchx)
            batchesy.append(batchy)
            batchesmasks.append(masks)
            batchx = []
            batchy = []
    return [batchesx, batchesy, batchesmasks]

# real_data is an array of strings containing multiple lines, each line is used as its own sample later in the script
real_data = list(set(flatten([x.split("\n") for x in open("./output_lt_20.txt", "r").read().split("\n\n")])))
# print(len(real_data))
# print(real_data)
random.shuffle(real_data)

pool = mp.Pool()
tokenized = list(tqdm(pool.imap(tokenize, real_data), total=len(real_data)))
# print(tokenized)

inputs = tokenized
outputs = list(tqdm(pool.imap(process, tokenized), total=len(tokenized)))
outputs.sort(key=lambda x: len(x[0]))
# print(outputs)

inputs = [[101] + x[0] + [102] for x in outputs]
outputs = [[-100] + x[1] + [-100] for x in outputs]

batched = batcher(inputs, outputs)
pickle.dump(batched, open("./video_review_data.pkl", "wb"))

100%|██████████| 3965/3965 [00:00<00:00, 13984.73it/s]


In [ ]:
import random
import pickle
import numpy as np
import torch, gc
gc.collect()
torch.cuda.empty_cache()
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import pytorch_lightning as pl
from transformers import BertForMaskedLM

class PTDataset(Dataset):
    def __init__(self):
        self.data = pickle.load(open("./video_review_data.pkl", "rb"))
        idxs = list(range(len(self.data[0])))
        random.shuffle(idxs)
        self.data = [[x[i] for i in idxs] for x in self.data]
        print("Loaded {} rows of data".format(len(self.data[0])))
        print(self.data[0])

    def __len__(self):
        return len(self.data[0])

    def __getitem__(self, idx):
        return [torch.tensor(x[idx]) for x in self.data]


class BertLyricsPT(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.bert = BertForMaskedLM.from_pretrained("bert-large-uncased")

    def forward(self, x):
        # print("XXXXXXXXXXXXXXXXXX")
        # print(type(x))
        print(x)
        # print(len(x))
        # print("YYYYYYYYYYYYYYY")
        inp, out, mask = x
        # print("INP")
        # print(type(inp))
        # print(inp[0])
        #map(torch.stack, zip(*x)) #[t.size() for t in x] #x #[ torch.tensor(y) for y in x ]
        enc = self.bert(input_ids=inp[0], attention_mask=mask[0], labels=out[0])
        return enc

    def training_step(self, batch, batch_nb):
        yhat = self(batch)
        loss = yhat.loss
        return {'loss': loss}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=5e-5, eps=1e-4)


def collate_fn(data):
    print("Inside Collate fn")
    print(type(data))
    print(len(data))
    inp, out, mask = data
    zipped = zip(inp, out, mask)
    return list(zipped)

def collate_fn2(batch):
    return tuple(zip(*batch))

def collate_fn3(data):
    bbox = data
    zipped = zip(bbox)
    return list(zipped)

if __name__ == "__main__":
    trainer = pl.Trainer(gpus=1, distributed_backend='ddp', max_epochs=4)

    dataset = PTDataset()
    dataloader = DataLoader(dataset, batch_size=None, collate_fn=collate_fn3) #num_workers=2, batch_size=16, 
 
    print("Loading model")
    model = BertLyricsPT()
    print("Loaded model")

    trainer.fit(model, dataloader)
    model.bert.save_pretrained("bert_video_review_pretrained")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


Loaded 30 rows of data
[[[101, 2017, 103, 12200, 2009, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2057, 2031, 103, 2341, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2026, 2240, 5881, 2098, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1045, 17010, 2008, 103, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 103, 2052, 2050, 11445, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 103, 1997, 1996, 2051, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2061, 4957, 2091, 103, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1045, 2066, 2502, 103, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 103, 2401, 7287, 2003, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 103, 13855, 2038, 2416, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2004, 2271, 103, 2748, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2017, 2156, 103, 2182, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 26126, 1045, 2469, 3246, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1045, 103, 2875, 2009, 102, 0, 0, 0, 0,

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/1


Loaded model



  | Name | Type            | Params
-----------------------------------------
0 | bert | BertForMaskedLM | 335 M 
-----------------------------------------
335 M     Trainable params
0         Non-trainable params
335 M     Total params
1,340.698 Total estimated model params size (MB)


[(tensor([[ 101, 2003, 2009,  ...,    0,    0,    0],
        [ 101, 1045, 2031,  ...,    0,    0,    0],
        [ 101,  103, 2003,  ...,    0,    0,    0],
        ...,
        [ 101, 2054, 2008,  ...,    0,    0,    0],
        [ 101, 2023, 2003,  ...,    0,    0,    0],
        [ 101, 2292,  103,  ...,    0,    0,    0]], device='cuda:0'),), (tensor([[-100, -100, -100,  ...,    0,    0,    0],
        [-100, -100, -100,  ...,    0,    0,    0],
        [-100, 2023, -100,  ...,    0,    0,    0],
        ...,
        [-100, -100, -100,  ...,    0,    0,    0],
        [-100, -100, -100,  ...,    0,    0,    0],
        [-100, -100, 1005,  ...,    0,    0,    0]], device='cuda:0'),), (tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'),)]
[(tensor([[  101,   102,     0,  ...,     0,     0,     0],
      

In [1]:
import sys
from transformers import BertTokenizer, BertForMaskedLM
import torch


tok = BertTokenizer.from_pretrained("bert-large-uncased")
model = BertForMaskedLM.from_pretrained("bert-large-uncased" if sys.argv[1] == "orig" else "bert_video_review_pretrained")

x = tok.encode("let's open this [MASK]")
idx = x.index(103)
x = torch.Tensor([x]).long()

l = model(x)[0]
l = l[0][idx].detach().cpu().numpy()
la = l.argsort()

for i in range(1, 11):
    print(tok.decode([la[-i]]))

up
one
door
now
tablet
here
thing
right
side
box
